# Exercise 6
***Andrés Mauricio Obando Acevedo***

## SVM & Regularization


For this homework we consider a set of observations on a number of red and white wine varieties involving their chemical properties and ranking by tasters. Wine industry shows a recent growth spurt as social drinking is on the rise. The price of wine depends on a rather abstract concept of wine appreciation by wine tasters, opinion among whom may have a high degree of variability. Pricing of wine depends on such a volatile factor to some extent. Another key factor in wine certification and quality assessment is physicochemical tests which are laboratory-based and takes into account factors like acidity, pH level, presence of sugar and other chemical properties. For the wine market, it would be of interest if human quality of tasting can be related to the chemical properties of wine so that certification and quality assessment and assurance process is more controlled.

Two datasets are available of which one dataset is on red wine and have 1599 different varieties and the other is on white wine and have 4898 varieties. All wines are produced in a particular area of Portugal. Data are collected on 12 different properties of the wines one of which is Quality, based on sensory data, and the rest are on chemical properties of the wines including density, acidity, alcohol content etc. All chemical properties of wines are continuous variables. Quality is an ordinal variable with possible ranking from 1 (worst) to 10 (best). Each variety of wine is tasted by three independent tasters and the final rank assigned is the median rank given by the tasters.

A predictive model developed on this data is expected to provide guidance to vineyards regarding quality and price expected on their produce without heavy reliance on volatility of wine tasters.

In [2]:
import pandas as pd
import numpy as np

In [3]:
data_r = pd.read_csv('https://github.com/albahnsen/PracticalMachineLearningClass/raw/master/datasets/Wine_data_red.csv')
data_w = pd.read_csv('https://github.com/albahnsen/PracticalMachineLearningClass/raw/master/datasets/Wine_data_white.csv')

In [4]:
data = data_w.assign(type = 'white')

data = data.append(data_r.assign(type = 'red'), ignore_index=True)
data.sample(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
879,6.9,0.12,0.36,2.2,0.037,18.0,111.0,0.99190,3.41,0.82,11.9,8,white
2896,7.0,0.31,0.39,7.5,0.055,42.0,218.0,0.99652,3.37,0.54,10.3,5,white
5821,6.8,0.41,0.31,8.8,0.084,26.0,45.0,0.99824,3.38,0.64,10.1,6,red
2824,7.2,0.26,0.40,6.3,0.047,52.0,172.0,0.99573,3.18,0.53,9.5,6,white
1202,6.4,0.30,0.38,7.8,0.046,35.0,192.0,0.99550,3.10,0.37,9.0,5,white


# Exercise 6.1

Show the frecuency table of the quality by type of wine

Se busca clasificar cuales son buenos y malos en binario

In [5]:
data[['quality','type']].groupby('quality').count()

,type
quality,
3,30
4,216
5,2138
6,2836
7,1079
8,193
9,5


# SVM

# Exercise 6.2

* Standarized the features (not the quality)
* Create a binary target for each type of wine
* Create two Linear SVM's for the white and red wines, repectively.


before aply a stadarized model, is important to know the distribution of data, let's see:

In [6]:
hist = data.hist(bins=9, layout=(3,4),figsize=(15,10))

We can observe on histograms, that the data are in differente range each one, some are in decimals, others in units or hundreds; but, there are not values with frecuencies anormaly, so it's possible and necesary stadarize the features with the scope of give all the same weight with independency of their magnitude.

**Standarize**

In [7]:
from sklearn.preprocessing import Normalizer
X = data.drop(['quality','type'],axis=1)
transformer = Normalizer().fit(X)
X_normalized=transformer.transform(X)

In [8]:
X_norm = pd.DataFrame(columns=X.columns, data=X_normalized)
X_norm.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,0.039447,0.001522,0.002029,0.116649,0.000254,0.253585,0.957988,0.005641,0.016906,0.002536,0.049590
1,0.047267,0.002251,0.002551,0.012004,0.000368,0.105038,0.990361,0.007458,0.024759,0.003676,0.071276
2,0.078912,0.002728,0.003897,0.067221,0.000487,0.292266,0.944992,0.009694,0.031760,0.004287,0.098396
3,0.037412,0.001195,0.001663,0.044166,0.000301,0.244214,0.966464,0.005173,0.016575,0.002078,0.051441
4,0.037412,0.001195,0.001663,0.044166,0.000301,0.244214,0.966464,0.005173,0.016575,0.002078,0.051441


In [9]:
data2=pd.concat([X_norm,data[['quality','type']]], axis=1,join='inner')
data2.sample(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
1558,0.053279,0.003695,0.004211,0.054997,0.000387,0.103120,0.988232,0.008562,0.028100,0.004898,0.077340,4,white
3318,0.060468,0.001466,0.003207,0.016491,0.000385,0.238207,0.961991,0.009067,0.029226,0.006871,0.113607,7,white
2862,0.055528,0.001198,0.003484,0.017421,0.000305,0.130655,0.979911,0.010780,0.038870,0.005662,0.132832,6,white
3811,0.045486,0.001769,0.002401,0.012635,0.000329,0.145304,0.985541,0.006262,0.019774,0.003285,0.070125,5,white
4975,0.190304,0.021969,0.000000,0.067166,0.002911,0.391801,0.839575,0.027891,0.098510,0.015392,0.299448,6,red


**Binary Target**

In [10]:
data2['quality2'] = data2['quality'] > 6

In [11]:
data_white=data2[data2['type']=='white']
data_red=data2[data2['type']=='red']
data_white.shape,data_red.shape

((4898, 14), (1599, 14))

**Linear SVM**

In [12]:
from sklearn.svm import SVC # "Support Vector Classifier"
y_w = data_white['quality2']
X_w = data_white.drop(['quality','type'],axis=1)

y_r = data_red['quality2']
X_r = data_red.drop(['quality','type'],axis=1)

**_White wine_**

In [13]:
clf_WW = SVC(kernel='linear')
clf_WW.fit(X_w, y_w)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [14]:
y_w_pred = clf_WW.predict(X_w)

**_Red wine_**

In [15]:
clf_rw = SVC(kernel='linear')
clf_rw.fit(X_r, y_r)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [16]:
y_r_pred = clf_WW.predict(X_r)

# Exercise 6.3

Test the two SVM's using the different kernels (‘poly’, ‘rbf’, ‘sigmoid’)


In [17]:
from sklearn.metrics import accuracy_score

Accuracy for linear SVM

In [18]:
w_acc_lin=accuracy_score(y_w, y_w_pred, normalize=True)
r_acc_lin=accuracy_score(y_r, y_r_pred, normalize=True)
w_acc_lin,r_acc_lin

(1.0, 1.0)

Now, let' try with differents kernels and compare the accuracy of each.

_White wine:_

In [19]:
kernels=['linear','poly', 'rbf', 'sigmoid']
accuracyname=['A_w_poly', 'A_w_rbf', 'A_w_sigmoid']
Acc_val = []
for k in range(len(kernels)):
    clf_WW_r = SVC(kernel=kernels[k])
    clf_WW_r.fit(X_w, y_w)
    y_w_pred_r = clf_WW_r.predict(X_w)
    Acc_val.append(accuracy_score(y_w, y_w_pred_r, normalize=True))
    print('WW_Acc_'+kernels[k]+':',Acc_val[k])

WW_Acc_linear: 1.0
WW_Acc_poly: 1.0
WW_Acc_rbf: 1.0
WW_Acc_sigmoid: 1.0


In [20]:
from sklearn.metrics import f1_score

In [21]:
Acc_val = []
for k in range(len(kernels)):
    clf_WW_r = SVC(kernel=kernels[k])
    clf_WW_r.fit(X_w, y_w)
    y_w_pred_r = clf_WW_r.predict(X_w)
    Acc_val.append(f1_score(y_w, y_w_pred_r, average='binary'))
    print('WW_f1_'+kernels[k]+':',Acc_val[k])

WW_f1_linear: 1.0
WW_f1_poly: 1.0
WW_f1_rbf: 1.0
WW_f1_sigmoid: 1.0


_Red wine:_

In [22]:
Acc_val = []
for k in range(len(kernels)):
    clf_RW_r = SVC(kernel=kernels[k])
    clf_RW_r.fit(X_w, y_w)
    y_r_pred_r = clf_RW_r.predict(X_r)
    Acc_val.append(accuracy_score(y_r, y_r_pred_r, normalize=True))
    print('RW_Acc_'+kernels[k]+':',Acc_val[k])

RW_Acc_linear: 1.0
RW_Acc_poly: 1.0
RW_Acc_rbf: 1.0
RW_Acc_sigmoid: 1.0


# Exercise 6.4
Using the best SVM find the parameters that gives the best performance

'C': [0.1, 1, 10, 100, 1000], 'gamma': [0.01, 0.001, 0.0001]

Se evalua con la clasificación: el accuracy

In [27]:
kernels='rbf'
C = [0.1, 1, 10, 100, 1000]
gamma = [0.01, 0.001, 0.0001]

In [28]:
k=0
#White Wine
for i in range(len(C)):
    for j in range(len(gamma)):
        clf_WW_r = SVC(kernel=kernels, C=C[i], gamma=gamma[j])
        clf_WW_r.fit(X_w, y_w)
        y_w_pred_r = clf_WW_r.predict(X_w)
        Acc_val.append(accuracy_score(y_w, y_w_pred_r, normalize=True))
        print('WW_f1_'+str(C[i])+'_'+str(gamma[j])+':',Acc_val[k])
        k+=k

WW_f1_0.1_0.01: 1.0
WW_f1_0.1_0.001: 1.0
WW_f1_0.1_0.0001: 1.0
WW_f1_1_0.01: 1.0
WW_f1_1_0.001: 1.0
WW_f1_1_0.0001: 1.0
WW_f1_10_0.01: 1.0
WW_f1_10_0.001: 1.0
WW_f1_10_0.0001: 1.0
WW_f1_100_0.01: 1.0
WW_f1_100_0.001: 1.0
WW_f1_100_0.0001: 1.0
WW_f1_1000_0.01: 1.0
WW_f1_1000_0.001: 1.0
WW_f1_1000_0.0001: 1.0


In [29]:
k=0
#Red Wine
for i in range(len(C)):
    for j in range(len(gamma)):
        clf_WW_r = SVC(kernel=kernels, C=C[i], gamma=gamma[j])
        clf_WW_r.fit(X_w, y_w)
        y_w_pred_r = clf_WW_r.predict(X_w)
        Acc_val.append(accuracy_score(y_w, y_w_pred_r, normalize=True))
        print('RW_f1_'+str(C[i])+'_'+str(gamma[j])+':',Acc_val[k])
        k+=k

RW_f1_0.1_0.01: 1.0
RW_f1_0.1_0.001: 1.0
RW_f1_0.1_0.0001: 1.0
RW_f1_1_0.01: 1.0
RW_f1_1_0.001: 1.0
RW_f1_1_0.0001: 1.0
RW_f1_10_0.01: 1.0
RW_f1_10_0.001: 1.0
RW_f1_10_0.0001: 1.0
RW_f1_100_0.01: 1.0
RW_f1_100_0.001: 1.0
RW_f1_100_0.0001: 1.0
RW_f1_1000_0.01: 1.0
RW_f1_1000_0.001: 1.0
RW_f1_1000_0.0001: 1.0


# Exercise 6.5

Compare the results with other methods

Se compara con LOGIT

In [30]:
from sklearn.linear_model import LogisticRegression

In [33]:
#White wine
logreg_w = LogisticRegression(solver='liblinear',C=1e9)
logreg_w.fit(X_w, y_w)
y_w_pred_log = logreg_w.predict(X_w)
accuracy_score(y_w, y_w_pred_log, normalize=True)

1.0

In [34]:
#Red wine
logreg_r = LogisticRegression(solver='liblinear',C=1e9)
logreg_r.fit(X_r, y_r)
y_r_pred_log = logreg_r.predict(X_r)
accuracy_score(y_r, y_r_pred_log, normalize=True)

1.0

# Regularization

# Exercise 6.6


* Train a linear regression to predict wine quality (Continous) (Lo mismo del 6.5)

* Analyze the coefficients

* Evaluate the RMSE

In [35]:
logreg_w.coef_

array([[-3.03245868e-01, -1.46716163e-02, -1.37582577e-02,
        -4.30390656e-01, -3.51321287e-03, -9.48259414e-01,
        -6.00373771e+00, -3.93635881e-02, -1.17436837e-01,
        -1.43747951e-02, -2.03479782e-01,  2.43991782e+01]])

In [36]:
logreg_r.coef_

array([[-1.19582235e+00, -2.00433779e-01,  3.45562100e-02,
        -2.85356380e-01, -2.21512409e-02, -1.25691944e+00,
        -5.60473090e+00, -1.86696350e-01, -6.39977732e-01,
        -3.43681327e-02, -1.25035218e+00,  2.42871600e+01]])

**RMSE**

In [42]:
from sklearn import metrics
y_tr=[y_w]
y_pr=[y_w_pred_log]
MSE_logreg_w = metrics.mean_squared_error(y_tr, y_pr)
print('RMSE:', np.sqrt(MSE_logreg_w))

TypeError: numpy boolean subtract, the `-` operator, is deprecated, use the bitwise_xor, the `^` operator, or the logical_xor function instead.

# Exercise 6.7

* Estimate a ridge regression with alpha equals 0.1 and 1.
* Compare the coefficients with the linear regression
* Evaluate the RMSE

# Exercise 6.8

* Estimate a lasso regression with alpha equals 0.01, 0.1 and 1.
* Compare the coefficients with the linear regression
* Evaluate the RMSE

# Exercise 6.9

* Create a binary target

* Train a logistic regression to predict wine quality (binary)

* Analyze the coefficients

* Evaluate the f1score

# Exercise 6.10

* Estimate a regularized logistic regression using:
* C = 0.01, 0.1 & 1.0
* penalty = ['l1, 'l2']
* Compare the coefficients and the f1score

Usando regularización